In [ ]:
import pandas as pd
import numpy as np
import os
from gensim.models.word2vec import Word2Vec

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dfselect = pd.read_csv('select_name.csv') 
# this file only contains 12470 files which was filtered by cosine similarity
# you can replace with your only csv with (1) the article and (2) the word list after cut word
dflabel = pd.read_csv('TrainLabel.csv')

w2vmodel = Word2Vec.load('w2v50.model') # load the cut word w2v model to find the index of the word

In [ ]:
# in next sentence prediction model, the relative sentence is labeled with 0, 
# not relative sentence is labeled with 1
# use the same label in classification

labeltable = np.ones((1402, 1402))
for i in range(len(dflabel)):
    labeltable[dflabel.iloc[i,0], dflabel.iloc[i,1]] = 0

In [ ]:
# preprocess the ids instead of using the tokenizer of BERT
# cut the words first, and replace the words with the index in the word dictionary 
# the dictionary contains all words in both Train dataset and Public dataset

Path = './Train_only/'
filelist = os.listdir(Path)

sentence = []; label = []
maxl = 0
for i in range(len(dfselect)):
    label.append(labeltable[dfselect.iloc[i,0], dfselect.iloc[i,1]])
    temp = [101] # the start of the sentence
    for j in range(0,2):
        f = np.load(Path+str(dfselect.iloc[i,j]).zfill(4)+'.npz')
        data1= list(f['text'])
        for word in data1:
            if len(word) > 1 and (w2vmodel.wv.index_to_key.index(word)+105 not in temp):
                temp.append(w2vmodel.wv.index_to_key.index(word)+105)
        temp = temp+[102] # the end of the each sentence
        if len(temp)>256 and j==0: # if the first sentence is too long, cut it!
            print(j, len(temp))
            temp = temp[:256]+[102] 
            
    if len(temp) > 512: # if the second sentence is too long, cut it!
        print(len(temp))
        temp = temp[0:511]+[102]
        
    sentence.append(temp)


In [ ]:
len(label)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN=512

input_ids=pad_sequences(sentence, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print("Padding 第一個句子:",input_ids[0])
#建立mask
attention_mask = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_mask.append(seq_mask)
    
print("第一個attention mask:",attention_mask[0])

In [ ]:
np.savez('BertClassTrain', ids = input_ids, l = label, mask = attention_mask)

In [ ]:
# repeat the steps to deal with unbalence problem

dflabel = pd.read_csv('TrainLabel.csv')
dfselect = pd.read_csv('TrainLabel.csv')
Path = './Train_only/'
filelist = os.listdir(Path)

sentence = []; label = []
maxl = 0
for i in range(len(dfselect)):
    label.append(labeltable[dfselect.iloc[i,0], dfselect.iloc[i,1]])
    temp = [101] # the start of the sentence
    for j in range(0,2):
        f = np.load(Path+str(dfselect.iloc[i,j]).zfill(4)+'.npz')
        data1= list(f['text'])
        for word in data1:
            if len(word) > 1 and (w2vmodel.wv.index_to_key.index(word)+105 not in temp):
                temp.append(w2vmodel.wv.index_to_key.index(word)+105)
        temp = temp+[102] # the end of the each sentence
        if len(temp)>256 and j==0: # if the first sentence is too long, cut it!
            print(j, len(temp))
            temp = temp[:256]+[102] 
            
    if len(temp) > 512: # if the second sentence is too long, cut it!
        print(len(temp))
        temp = temp[0:511]+[102]
        
    sentence.append(temp)


In [ ]:
# copy the relative data to balance the non-relative data
# 1382*(2^3) = 1381*8 = 11048
for i in range(0, 3):
    sentence += sentence
    label += label

In [ ]:
len(label)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN=512

input_ids=pad_sequences(sentence, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print("Padding 第一個句子:",input_ids[0])
#建立mask
attention_mask = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_mask.append(seq_mask)
    
print("第一個attention mask:",attention_mask[0])

In [ ]:
np.savez('BertClassTrainTrue', ids = input_ids, l = label, mask = attention_mask)

In [ ]:
import pandas as pd
import os
import numpy as np
from transformers import BertTokenizer, BertModel, AdamW
from transformers import BertForNextSentencePrediction, BertForSequenceClassification
from torchsummary import summary
import torch
from torch import nn

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese')
# model = torch.load('bertNextSeq1204_20.pkl')

In [ ]:
import numpy as np

f = np.load('BertClassTrain.npz')
input_ids1 = f['ids']
label1 = f['l']
attention_mask1 = f['mask']

f = np.load('BertClassTrainTrue.npz')
input_ids = f['ids']
label = f['l']
attention_mask = f['mask']

In [ ]:
np.shape(input_ids)

In [ ]:
# concate data and shuffle it

import random

index = list(range(0,len(label)+len(label1)))
random.shuffle(index)
sinput_ids = np.concatenate((input_ids1, input_ids), axis=0)
slabel = np.concatenate((label1, label), axis=0)
sattention_mask = np.concatenate((attention_mask1, attention_mask), axis=0)

In [ ]:
len(index)

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Batsize = 4 # depends on your gpu memory

train_inputs = torch.tensor(sinput_ids[index[3200:]])
validation_inputs = torch.tensor(sinput_ids[index[:3200]])

train_labels = torch.tensor(slabel[index[3200:]], dtype=torch.long)
validation_labels = torch.tensor(slabel[index[:3200]], dtype=torch.long)

train_masks = torch.tensor(sattention_mask[index[3200:]])
validation_masks = torch.tensor(sattention_mask[index[:3200]])

print(np.shape(train_inputs), np.shape(train_labels), np.shape(train_masks))
print(np.shape(validation_inputs), np.shape(validation_labels), np.shape(validation_masks))

train_data1 = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler1 = RandomSampler(train_data1)
train_dataloader1 = DataLoader(train_data1, sampler=train_sampler1, batch_size=Batsize)

va_data1 = TensorDataset(validation_inputs, validation_masks, validation_labels)
va_sampler1 = RandomSampler(va_data1)
va_dataloader1 = DataLoader(va_data1, sampler=va_sampler1, batch_size=Batsize)

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [ ]:
model

In [ ]:
from tqdm.notebook import tqdm
import time

l1loss = torch.nn.L1Loss()

acc_train = []
epochs = 100
model.train().cuda()

for epoch in range(epochs):
    avg_loss = 0; acc = 0; va_avg_loss = 0; va_acc = 0;
    
    for step, batch1 in enumerate(train_dataloader1): 
        batch1 = tuple(t.cuda() for t in batch1)
        b_input_ids1, b_input_mask1, b_labels1 = batch1
        optimizer.zero_grad()
        cross, logit = model(b_input_ids1, token_type_ids=None, attention_mask=b_input_mask1, labels=b_labels1)[0:2]
        acc += l1loss(torch.argmax(logit, dim=1).float(), b_labels1)
        cross.backward()        # retain_graph=True 
        optimizer.step()

        print(step, 'Inst_Loss: ', '%.6f' % float(cross), ', ACC: ', '%d' % float(acc), end = '\r')
        avg_loss += float(cross)
    
    acc_train.append(acc)    
    now = time.asctime(time.localtime(time.time()))  
    print('Epoch:', '%3d' % epoch, ', AVG loss:', '%.6f' % avg_loss , ', ACC:', '%d' % float(acc),', Time:', now)    
    
    model.eval()
    for step, batch1 in enumerate(va_dataloader1): 
        batch1 = tuple(t.cuda() for t in batch1)
        b_input_ids1, b_input_mask1, b_labels1 = batch1
        cross, logit = model(b_input_ids1, token_type_ids=None, attention_mask=b_input_mask1, labels=b_labels1)[0:2]
        va_acc += l1loss(torch.argmax(logit, dim=1).float(), b_labels1)

        print(step, 'va_Loss: ', '%.6f' % float(cross), ', ACC: ', '%d' % float(va_acc), end = '\r')
        va_avg_loss += float(cross)
    model.train()
    
    now = time.asctime(time.localtime(time.time()))  
    print('Epoch:', '%3d' % epoch, ', AVG VA loss:', '%.6f' % va_avg_loss , ', ACC:', '%d' % float(va_acc),', Time:', now)
    
    if (epoch+1)% 5 == 0:
        torch.save(model, 'BertSequenceClassification1206_'+str(epoch+1).zfill(2)+'.pkl') 

In [ ]:
# Please restart the kernel 
# Use the trained model to predict

import pandas as pd
import os
import numpy as np
from transformers import BertTokenizer, BertModel, AdamW
from torchsummary import summary
from tqdm import tqdm_notebook as tqdm
import torch
from torch import nn

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

epoch = 25# which epoch you want to use
model = torch.load('BertSequenceClassification1206_'+str(epoch)+'.pkl')

In [ ]:
import pandas as pd
import numpy as np
import os
from gensim.models.word2vec import Word2Vec
size = 50

dfselect = pd.read_csv('public_select1204.csv') # this file contain 17343 data which filtered by cosine similarity

w2vmodel = Word2Vec.load('w2v'+str(size)+'.model')

Path = './Public_only/'
filelist = os.listdir(Path)

sentence = []; label = []

for i in range(len(dfselect)):
    temp = [101]
    for j in range(0,2):
        f = np.load(Path+str(dfselect.iloc[i,j]).zfill(4)+'.npz')
        data1= list(f['text'])
        for word in data1:
            if len(word) > 1 and (w2vmodel.wv.index_to_key.index(word)+105 not in temp):
                temp.append(w2vmodel.wv.index_to_key.index(word)+105)
        temp = temp+[102]
        if len(temp)>256 and j==0:
            temp = temp[:256]+[102]
    
    if len(temp) > 512:
        temp = temp[0:511]+[102]
    sentence.append(temp)


In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN=512

input_ids=pad_sequences(sentence, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print("Padding 第一個句子:",input_ids[0][:256])
#建立mask
attention_mask = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_mask.append(seq_mask)


In [ ]:
np.savez('ClassPublic', ids = input_ids, mask = attention_mask)

In [ ]:
import numpy as np

f = np.load('ClassPublic.npz')
input_ids = f['ids']
attention_mask = f['mask']


In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_mask)
print(np.shape(inputs), np.shape(masks))

public_data1 = TensorDataset(inputs, masks)
dataloader1 = DataLoader(public_data1, shuffle=False, batch_size=1) 
# prediction 1 by 1, and do not shuffle, otherwise we will miss the order in the csv file

In [ ]:
model.eval()
ans = []; i = 0;
for step, batch1 in enumerate(dataloader1): 
    batch1 = tuple(t.cuda() for t in batch1)
    b_input_ids1, b_input_mask1= batch1
    out = model(b_input_ids1, token_type_ids=None, attention_mask=b_input_mask1)[0]
    i+=1
    if out[0][0] > out[0][1]:
        ans.append(i)
        print(i)

In [ ]:
dfselect.iloc[ans].to_csv('1206_class.csv', index=False) 